In [2]:
import numpy as np

In [3]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10 
NUM_DIMENSIONS = 2 

In [4]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [5]:
CONSTRAINTS

array([[ 78, 205],
       [315, 115],
       [201, 298]])

In [ ]:
import numpy as np
from icecream import ic
from tqdm import trange 

def cost(sol):
    # checks each column (each item) across all knapsacks
    # returns a boolean vector of length NUM_ITEMS, where each entry is True if that item appears in at least one knapsack
    # and sum values of selected items
    return VALUES[np.any(sol, axis=0)].sum()

def is_feasible(total_weights):
    return not np.any(total_weights > CONSTRAINTS)

def random_solution():
    # this approach mantains incremental weights 
    sol = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
    total_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))
    for i in np.random.permutation(NUM_ITEMS):
        k = np.random.randint(NUM_KNAPSACKS)
        if np.all(total_weights[k] + WEIGHTS[i] <= CONSTRAINTS[k]):
            sol[k, i] = True # add item to solution
            total_weights[k] += WEIGHTS[i] # update incremental weights
    value = cost(sol)
    return sol, total_weights, value

def simulated_annealing(max_iter=20000, T0=1000.0, alpha=0.999):
    sol, total_weights, current_val = random_solution()
    best_sol = sol.copy()
    best_val = current_val
    T = T0

    for it in trange(max_iter, desc="Simulated Annealing"):
        # pick random item 
        i = np.random.randint(NUM_ITEMS)
        # look if the item is already in a knapsack (else None)
        old_k = np.where(sol[:, i])[0]
        old_k = old_k[0] if len(old_k) > 0 else None
        # pick a new knapsack (-1 = remove from solution)
        k = np.random.randint(-1, NUM_KNAPSACKS)
        if k == old_k: # skip if there is no actual change (i.e. moving same item in the same knapsack)
            continue
        
        # check feasibility
        new_weights = total_weights.copy()
        if old_k is not None:
            new_weights[old_k] -= WEIGHTS[i] # remove weights from old knapsack
        if k != -1:
            new_weights[k] += WEIGHTS[i] # add weights to new knapsack
        if not is_feasible(new_weights):
            continue  # infeasible -> skip 
            
        # delta cost
        delta = VALUES[i] * ((k != -1) - (old_k is not None))
        # take item : VALUES[i] * (1-0)   = +VALUES[i]
        # remove item : VALUES[i] * (0-1)    = -VALUES[i]
        # swap two items : VALUES[i] * (1-1) = 0
        new_val = current_val + delta
        if delta > 0 or np.random.random() < np.exp(delta / T):
            # accept solution and apply move
            if old_k is not None:
                sol[old_k, i] = False # remove item from old knapsack
            if k != -1:
                sol[k, i] = True # add item to new knapsack
            total_weights = new_weights
            current_val = new_val
            if new_val > best_val:
                best_val = new_val
                best_sol = sol.copy()
        
        T *= alpha
        if T < 1e-3:
            break

    return best_sol, best_val

## TEST PROBLEMS

In [12]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
max_theoretical = VALUES.sum()
print("theoretical max:", max_theoretical)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = simulated_annealing()
ic(best_sol.astype(int))
ic(best_val)

theoretical max: 1065


Simulated Annealing: 100%|██████████| 20000/20000 [00:00<00:00, 101439.96it/s]
ic| best_sol.astype(int): array([[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
                                 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]])
ic| best_val: 1065


1065

In [22]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
max_theoretical = VALUES.sum()
print("theoretical max:", max_theoretical)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = simulated_annealing(max_iter=100_000)
# ic(best_sol.astype(int))
ic(best_val)

theoretical max: 52620


Simulated Annealing: 100%|██████████| 100000/100000 [00:00<00:00, 116194.20it/s]
ic| best_val: 49017


49017

In [32]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
max_theoretical = VALUES.sum()
print("theoretical max (without constraints):", max_theoretical)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = simulated_annealing(max_iter=500_000)
ic(best_sol.astype(int))
ic(best_val)

theoretical max (without constraints): 2490698


Simulated Annealing: 100%|██████████| 500000/500000 [00:08<00:00, 57174.93it/s]
ic| best_sol.astype(int): array([[0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0],
                                 ...,
                                 [0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0]])
ic| best_val: 1465272


1465272